In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.

# SA-Co/VEval データの視覚化 (Visualization)

このノートブックでは、SA-Co/VEvalデータセットの構造を理解し、アノテーションデータを視覚化する方法を示します。
データの読み込み、データ形式の確認、そして実際の画像とマスクの表示を行います。

## Google Colab セットアップ

Google Colabで実行する場合は、以下のセルを実行してください。
これにより、必要なライブラリのインストールと、データの読み込み準備が行われます。

In [ ]:
# Google Colab環境のセットアップ
try:
    import google.colab
    IN_COLAB = True
    print("Google Colab環境で実行中")
except ImportError:
    IN_COLAB = False
    print("ローカル環境で実行中")

if IN_COLAB:
    # 1. Google Driveをマウント (推奨)
    # 自分のデータやコードを使用する場合は、Google Driveにアップロードしてマウントします
    from google.colab import drive
    drive.mount('/content/drive')
    
    # 2. SAM3のインストールとパス設定
    import os
    import sys
    
    # Google Drive内のSAM3ディレクトリのパス
    # ※ご自身の環境に合わせてパスを変更してください
    DRIVE_SAM3_PATH = "/content/drive/MyDrive/sam3"
    
    if os.path.exists(DRIVE_SAM3_PATH):
        print(f"Google Drive内のSAM3が見つかりました: {DRIVE_SAM3_PATH}")
        os.chdir(DRIVE_SAM3_PATH)
        print("カレントディレクトリを変更しました。")
        
        # 依存関係のインストール
        print("依存関係をインストールしています...")
        # Numpy 2.0との互換性問題を回避するためにバージョンを固定
        !pip install -q "numpy<2.0"
        !pip install -q -e .
        !pip install -q pycocotools
        
    else:
        print(f"Google Drive内に {DRIVE_SAM3_PATH} が見つかりませんでした。")
        print("GitHubからSAM3をクローンしてインストールします...")
        
        # GitHubからクローン
        if not os.path.exists("/content/sam3"):
            !git clone https://github.com/facebookresearch/sam3.git /content/sam3
            
        os.chdir("/content/sam3")
        print("カレントディレクトリを /content/sam3 に変更しました。")
        
        # 依存関係のインストール
        # Numpy 2.0との互換性問題を回避するためにバージョンを固定
        !pip install -q "numpy<2.0"
        !pip install -q -e .
        !pip install -q pycocotools
    
    print("セットアップが完了しました。")
    print(f"現在の作業ディレクトリ: {os.getcwd()}")

### データの使用について

- **Google Driveを使用する場合**: `GT_DIR` や `PRED_DIR` には `/content/drive/MyDrive/...` から始まるパスを指定してください。
- **GitHubからクローンした場合**: 左側のファイルブラウザから `/content/sam3` 内を確認できます。データは別途アップロードが必要です。

In [ ]:
# Google Colab環境のセットアップ
try:
    import google.colab
    IN_COLAB = True
    print("Google Colab環境で実行中")
except ImportError:
    IN_COLAB = False
    print("ローカル環境で実行中")

if IN_COLAB:
    # 1. Google Driveをマウント (推奨)
    # 自分のデータやコードを使用する場合は、Google Driveにアップロードしてマウントします
    from google.colab import drive
    drive.mount('/content/drive')
    
    # 2. SAM3のインストールとパス設定
    import os
    import sys
    
    # Google Drive内のSAM3ディレクトリのパス
    # ※ご自身の環境に合わせてパスを変更してください
    DRIVE_SAM3_PATH = "/content/drive/MyDrive/sam3"
    
    if os.path.exists(DRIVE_SAM3_PATH):
        print(f"Google Drive内のSAM3が見つかりました: {DRIVE_SAM3_PATH}")
        os.chdir(DRIVE_SAM3_PATH)
        print("カレントディレクトリを変更しました。")
        
        # 依存関係のインストール
        print("依存関係をインストールしています...")
        !pip install -q -e .
        !pip install -q pycocotools
        
    else:
        print(f"Google Drive内に {DRIVE_SAM3_PATH} が見つかりませんでした。")
        print("GitHubからSAM3をクローンしてインストールします...")
        
        # GitHubからクローン
        if not os.path.exists("/content/sam3"):
            !git clone https://github.com/facebookresearch/sam3.git /content/sam3
            
        os.chdir("/content/sam3")
        print("カレントディレクトリを /content/sam3 に変更しました。")
        
        # 依存関係のインストール
        !pip install -q -e .
        !pip install -q pycocotools
    
    print("セットアップが完了しました。")
    print(f"現在の作業ディレクトリ: {os.getcwd()}")

### データの使用について

- **Google Driveを使用する場合**: `GT_DIR` や `PRED_DIR` には `/content/drive/MyDrive/...` から始まるパスを指定してください。
- **GitHubからクローンした場合**: 左側のファイルブラウザから `/content/sam3` 内を確認できます。データは別途アップロードが必要です。

## 環境設定

Google Colabを使用しているかどうかを設定し、必要なライブラリをインストールします。

In [ ]:
using_colab = False

In [ ]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib scikit-learn
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/sam3.git'

## ライブラリのインポート

データ処理と視覚化に必要なライブラリをインポートします。

In [ ]:
import os
from glob import glob

import numpy as np
import utils

from matplotlib import pyplot as plt

COLORS = utils.pascal_color_map()[1:]

## 1. データの読み込み (Load the data)

データセットのパスを指定し、アノテーションファイルを読み込みます。
ユーザーは `DATA_DIR` を自身の環境に合わせて変更する必要があります。

In [ ]:
# データパスの準備
DATA_DIR = "./sam3_saco_veval_data" # ここにデータパスを入力
ANNOT_DIR = os.path.join(DATA_DIR, "annotation")

# SACO/Vevalアノテーションファイルの読み込み
annot_file_list = glob(os.path.join(ANNOT_DIR, "*veval*.json"))
annot_dfs = utils.get_annot_dfs(file_list=annot_file_list)

読み込まれたアノテーションファイルを確認します。

In [ ]:
annot_dfs.keys()

## 2. データ形式の例 (Examples of the data format)

読み込んだデータの構造を確認します。
`saco_veval_yt1b_val` データセットを例として使用します。

In [ ]:
annot_dfs["saco_veval_yt1b_val"].keys()

### データセット情報 (Info)

データセットのメタデータを確認します。

In [ ]:
annot_dfs["saco_veval_yt1b_val"]["info"]

### ビデオ情報 (Videos)

データセットに含まれるビデオの情報を確認します。

In [ ]:
annot_dfs["saco_veval_yt1b_val"]["videos"].head(3)

### アノテーション情報 (Annotations)

個々のアノテーションデータを確認します。

In [ ]:
annot_dfs["saco_veval_yt1b_val"]["annotations"].head(3)

### カテゴリ情報 (Categories)

定義されているカテゴリを確認します。

In [ ]:
annot_dfs["saco_veval_yt1b_val"]["categories"].head(3)

### ビデオと名詞句のペア (Video-Noun Phrase Pairs)

ビデオとそれに関連付けられた名詞句(Noun Phrase)のペアを確認します。

In [ ]:
annot_dfs["saco_veval_yt1b_val"]["video_np_pairs"].head(3)

## 3. データの視覚化 (Visualize the data)

実際の画像とアノテーションマスクを視覚化して確認します。
ランダムにビデオと名詞句のペアを選択し、対応するフレームとマスクを表示します。

In [ ]:
# ターゲットデータセットの選択
target_dataset_name = "saco_veval_yt1b_val"

# 正のマスクレットを持つビデオ-名詞句ペアをランダムに選択して視覚化
df_pairs = annot_dfs[target_dataset_name]["video_np_pairs"]
df_positive_pairs = df_pairs[df_pairs.num_masklets > 0]
rand_idx = np.random.randint(len(df_positive_pairs))
pair_row = df_positive_pairs.iloc[rand_idx]
video_id = pair_row.video_id
noun_phrase = pair_row.noun_phrase
print(f"ランダムに選択されたビデオ-名詞句ペア: video_id={video_id}, noun_phrase={noun_phrase}")

def display_image_in_subplot(img, axes, row, col, title=""):
    axes[row, col].imshow(img)
    axes[row, col].set_title(title)
    axes[row, col].axis('off')

num_frames_to_show = 5  # データセットごとに表示するフレーム数
every_n_frames = 4  # 表示するフレームの間隔

fig, axes = plt.subplots(num_frames_to_show, 3, figsize=(15, 5 * num_frames_to_show))

for idx in range(0, num_frames_to_show):
    sampled_frame_idx = idx * every_n_frames
    print(f"フレーム {sampled_frame_idx} のアノテーションを読み込み中")
    # フレームと対応するマスク、名詞句を取得
    frame, annot_masks, annot_noun_phrases = utils.get_all_annotations_for_frame(
        annot_dfs[target_dataset_name], video_id=video_id, frame_idx=sampled_frame_idx, data_dir=DATA_DIR, dataset=target_dataset_name
    )
    # 選択された名詞句でマスクと名詞句をフィルタリング
    annot_masks = [m for m, np in zip(annot_masks, annot_noun_phrases) if np == noun_phrase]

    # フレームを表示
    display_image_in_subplot(frame, axes, idx, 0, f"{target_dataset_name} - {noun_phrase} - Frame {sampled_frame_idx}")

    # アノテーションマスクを表示
    if annot_masks is None:
        print(f"video_id {video_id} のフレーム {sampled_frame_idx} にマスクが見つかりません")
    else:
        # 白背景上に全てのマスクを表示
        all_masks = utils.draw_masks_to_frame(
            frame=np.ones_like(frame)*255, masks=annot_masks, colors=COLORS[: len(annot_masks)]
        )
        display_image_in_subplot(all_masks, axes, idx, 1, f"{target_dataset_name} - {noun_phrase} - Frame {sampled_frame_idx} - Masks")
        
        # フレーム上にマスクを重ねて表示
        masked_frame = utils.draw_masks_to_frame(
            frame=frame, masks=annot_masks, colors=COLORS[: len(annot_masks)]
        )
        display_image_in_subplot(masked_frame, axes, idx, 2, f"Dataset: {target_dataset_name} - {noun_phrase} - Frame {sampled_frame_idx} - Masks overlaid")

plt.tight_layout()
plt.show()